In [1]:
import pandas as pd
import tensorflow as tf
import numpy as ns
from env import *

In [2]:
class ohlc_rnn(object):
    def __init__(self, optimizer, inputs=8,depth=2, hidden=128, keep_prob=0.5):
        self.inputs = tf.placeholder(dtype=tf.float32, shape=[None, inputs])
        self.LSTM = []
        self.state = ()
        for i in range (0, depth):
            cell = tf.contrib.rnn.BasicLSTMCell(hidden, state_is_tuple=True, forget_bias=keep_prob, activation=tf.nn.relu)
            #cell = tf.nn.rnn_cell.DropoutWrapper(cell_0, output_keep_prob=keep_prob)
            self.LSTM.append(cell)
            self.state = self.state + (tf.placeholder(tf.float32, [1, cell.state_size.c]), 
                                       tf.placeholder(tf.float32, [1, cell.state_size.h]))
        m_cell = tf.contrib.rnn.MultiRNNCell(self.LSTM, state_is_tuple=True)
        
        self.init_state = self.state = m_cell.zero_state(1, tf.float32)
        self.rnn_in = tf.expand_dims(self.inputs, [0])
        r_output, self.state_out = tf.nn.dynamic_rnn(cell=m_cell, inputs=self.rnn_in, initial_state=self.state, time_major=False)
        d_output = tf.layers.dense(r_output, inputs)
        self.output = tf.reshape(d_output, shape=[-1, inputs])
        
        loss = 0
        self.next_ohlc = tf.placeholder(dtype=tf.float32, shape=[None, inputs])
        loss = tf.losses.mean_squared_error(labels=self.next_ohlc, predictions=self.output)
        self.cost = tf.reduce_mean(loss)*100
        
        self.grads = optimizer.compute_gradients(self.cost)
        self._train_op = optimizer.apply_gradients(self.grads)
        
    def run_rnn(self, inputs, sess, state):
        output, state_ = sess.run([self.output, self.state_out], feed_dict={self.inputs:inputs, self.state:state})
        return output, state_
            
    def train_rnn(self, day, sess):
        ohlc, n_ohlc = day.get_next_ohlc()
        self.predict, state, ret, train = sess.run([self.output, self.state_out, self.cost, self._train_op],
                            feed_dict={self.inputs:ohlc, self.next_ohlc:n_ohlc})
        #print(st)
        return ret
    
    def get_init_state(self, sess):
        return sess.run([self.init_state], feed_dict={self.state:self.init_state})

In [3]:
forex = forex_env()
optimizer = tf.train.AdamOptimizer(1e-9)
rnn = ohlc_rnn(optimizer=optimizer)
save = tf.train.Saver()
train = True

In [ ]:
i=0
with tf.Session() as sess:
    kevin = tf.summary.FileWriter("./graph/graph", sess.graph)
    state = sess.run([rnn.init_state])
    if (train):
        sess.run(tf.global_variables_initializer())
        while forex.check_if_done():
            day = forex.get_d_forex()
            loss = rnn.train_rnn(day, sess)
            if (i % 60) is 0:
                print(loss)
                save.save(sess, "graph/model.ckpt")
            i=i+1
        forex.reset()
        while forex.check_if_done():
            day = forex.get_d_forex()
            loss = rnn.train_rnn(day, sess)
            if (i % 20) is 0:
                print(loss)
                save.save(sess, "graph/model.ckpt")
            i=i+1
    else:
        save.restore(sess, "graph/model.ckpt")
    #forex.reset()
    day = forex.get_d_forex()
    inp, n = day.get_next_ohlc()
    print(rnn.run_rnn(inp, sess, state, n))
    print(n)
    kevin.close()

This was a linear regressor using LSTMs for the A3C algorithm. It was not really accurate though. I believe this is because I did not normalize my data. I would like to come back to this project and try again some day